In [11]:
import numpy as np # Library for vectorized data
import pandas as pd # Library for data analysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json # Library for JSON files
from geopy.geocoders import Nominatim # Converting an address into Latitude/Longitude values
import requests # Library for requests of data
from bs4 import BeautifulSoup # library to parse out HTML and XML documents
from pandas.io.json import json_normalize # Tranforming a JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # importing k-means from clustering stage
!pip install folium
import folium # Library for map-rendering
print("Libraries imported.")

Libraries imported.


In [12]:
TorontoData = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(TorontoData, 'html.parser')
PostalCodeList = []
BoroughList = []
NeighborhoodList = []

In [14]:
# Sorting data into their respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeList.append(cells[0].text)
        BoroughList.append(cells[1].text)
        NeighborhoodList.append(cells[2].text.rstrip('\n')) # Avoiding any new lines in neighborhood cell

In [21]:
# New Dataframe
Toronto_df = pd.DataFrame({"PostalCode": PostalCodeList, "Borough": BoroughList, "Neighborhood": NeighborhoodList})
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [27]:
# Get rid of 'Not assigned' boroughs
Toronto_df_DropNA = Toronto_df[Toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
Toronto_df_DropNA.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [30]:
# Grouping neighborhoods that are in the same borough
Toronto_df_Grouped = Toronto_df_DropNA.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [33]:
# For Neighborhoods that are "Not assigned", make the value the same as Borough
for index, row in Toronto_df_Grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]
Toronto_df_Grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [35]:
# Test dataframe, just to check it is all done to the requirements of the assessment.
column_names = ["PostalCode", "Borough", "Neighborhood"]
Test_df = pd.DataFrame(columns=column_names)

Test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]

for postcode in Test_list:
    Test_df = Test_df.append(Toronto_df_Grouped[Toronto_df_Grouped["PostalCode"]==postcode], ignore_index=True)
    
Test_df

,PostalCode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,Central Bay Street
1,M2H\n,North York\n,Hillcrest Village
2,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
3,M1J\n,Scarborough\n,Scarborough Village
4,M4G\n,East York\n,Leaside
5,M4M\n,East Toronto\n,Studio District
6,M1R\n,Scarborough\n,"Wexford, Maryvale"
7,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L\n,North York\n,Humber Summit
9,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har..."


In [36]:
# Number of rows of the cleaned dataframe
Toronto_df_Grouped.shape

(103, 3)